# Carga de información a la base de datos BBDD_Hoteles

En este documento realizamos la carga de los datos limpiados a la base de sql.

In [12]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import psycopg2 as ps

In [13]:
# Importamos el csv con la información de las reservas, los hoteles y los clientes
data = pd.read_csv("../data/reservas_hoteles_limpio.csv", parse_dates=['fecha_reserva', 'inicio_estancia', 'final_estancia'], dtype={'id_cliente': str, 'id_hotel': str})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    15000 non-null  datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  object        
 10  nombre_hotel     15000 non-null  object        
 11  ciudad           15000 non-null  object        
 12  precio_noche     15000 non-null  float64       
 13  estrellas        15000 non-null  float64       
dtypes: bool(1), datetime64[ns](3), float64

In [14]:
data[["id_hotel", "nombre_hotel"]]

,id_hotel,nombre_hotel
0,1,Hotel Monte Verde
1,2,Hotel Brisas del Mar
2,3,Hotel Camino del Sol
3,4,Hotel Puerta del Cielo
4,5,Hotel Encanto Real
...,...,...
14995,28,ibis Madrid Getafe
14996,26,ibis Madrid Calle Alcalá
14997,27,ibis budget Madrid Vallecas
14998,23,ibis budget Madrid Centro las Ventas


In [15]:
data["precio_noche"]

0        276.612381
1        275.225921
2        269.998444
3        280.151243
4        278.243996
            ...    
14995     71.000000
14996     99.000000
14997     80.000000
14998    108.000000
14999    136.000000
Name: precio_noche, Length: 15000, dtype: float64

In [16]:
# Importamos el csv con los eventos de Madrid sacados de la API
eventos = pd.read_csv("../data/eventos_madrid.csv", parse_dates=['fecha_inicio', 'fecha_fin'])
eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_evento  209 non-null    object        
 1   url_evento     209 non-null    object        
 2   codigo_postal  209 non-null    int64         
 3   direccion      197 non-null    object        
 4   horario        126 non-null    object        
 5   organizacion   198 non-null    object        
 6   fecha_inicio   209 non-null    datetime64[ns]
 7   fecha_fin      209 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 13.2+ KB


In [17]:
eventos[eventos["codigo_postal"] == 0]

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion,fecha_inicio,fecha_fin
2,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,NaN,NaN,NaN,2024-10-14,2025-06-30
12,Actividades mensuales de los centros de mayore...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,NaN,NaN,NaN,2024-02-01,2025-07-13
15,Apoyo a la autonomía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,NaN,NaN,NaN,2024-09-13,2025-06-20
17,ArTrics - Laboratorio de Creación 2024/25,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,NaN,NaN,NaN,2024-10-01,2025-07-31
28,Ciclismo por Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,NaN,00:00,NaN,2024-09-01,2025-06-30
54,Correr por Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,NaN,00:00,NaN,2024-09-01,2025-06-30
99,Laboratorios ciudadanos: Rescatando la memoria...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,NaN,10:00,Centro de Información y Educación Ambiental de...,2025-02-01,2025-03-30
115,Ludotecas en Centro Sociocultural Latina,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,NaN,NaN,NaN,2024-10-05,2025-06-28
123,Meditación Vipassana,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,NaN,09:45,NaN,2024-09-18,2025-06-18
135,Puntos de intercambio gratuitos de libros en l...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,NaN,NaN,NaN,2023-04-10,2029-06-30


## Inicialización de la conexión con la BBDD y el cursor

In [18]:
# Creamos la conexión a la base de datos
# Vamos a crear una conexión a la base de datos.
conn = ps.connect(
    dbname = "BBDD_Hoteles", # base a la que nos queremos conectar
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432" # puerto en el que s eencuentra postgres
)

In [19]:
# Creamos un cursor el cual nos va a permitir ejecutar querys.
cur = conn.cursor()

In [20]:
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

## Inserción de datos

In [21]:
# COmprobamos que no haya valores duplicados en ambos dataframes
# data = data.drop_duplicates()
data.duplicated().sum()

np.int64(0)

### Tabla ciudad

In [22]:
data_insert_ciudad = ["Madrid"]

In [23]:
# Creamos la query de inserción de los datos de ciudad
insert_query_ciudad = """ 
                        INSERT INTO ciudad (nombre_ciudad)
                        VALUES (%s) 
"""

In [24]:
# Ejecutamos la query y le indicamos que solo tenemos una ciudad que es Madrid
cur.execute(insert_query_ciudad, data_insert_ciudad)
conn.commit()

In [25]:
# vamos a comprobar que se ha creado correctamente
query_ciudad = """
    SELECT * 
    FROM ciudad; 
"""
cur.execute(query_ciudad)
cur.fetchall()

[(1, 'Madrid')]

# NO VOLVER A EJECUTAR LA QUERY DE CIUDAD, ESTA YA INSERTADA

### Tabla eventos

In [17]:
conn.rollback()

In [26]:
# Sacamos el id de ciudad de la tabla de ciudad
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
dictio_ciudad = dict(cur.fetchall())
dictio_ciudad

{'Madrid': 1}

In [27]:
# Creamos la lista que va a contener toda la información 
data_insert_eventos = []

# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in eventos.iterrows():
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"] if row["codigo_postal"] != 0 else None
    direccion = row["direccion"] if pd.notna(row["direccion"]) else None # tenemos valores nulos en esta columna, de manera que le indicamos, que si el elemento no es nulo nos lo coja, y si es nulo, nos incluya un None, ya que sql no acepta valores nulos que no sean None
    horario = row["horario"] if pd.notna(row["horario"]) else None
    fecha_inicio = row["fecha_inicio"]
    fecha_fin = row["fecha_fin"]
    organizacion = row["organizacion"] if pd.notna(row["organizacion"]) else None
    id_ciudad = dictio_ciudad.get("Madrid")

    # incluimos los valores en la lista
    data_insert_eventos.append((nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad))

In [28]:
data_insert_eventos

[('60 Premio Reina Sofía de Pintura y Escultura',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD',
  28009,
  'PASEO COLOMBIA 1',
  None,
  Timestamp('2025-02-27 00:00:00'),
  Timestamp('2025-03-23 00:00:00'),
  'Centro Cultural Casa de Vacas (Retiro)',
  1),
 ('A toda vela',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=59d5e7da8b822910VgnVCM1000001d4a900aRCRD',
  28045,
  'PLAZA LEGAZPI 8',
  None,
  Timestamp('2024-10-01 00:00:00'),
  Timestamp('2025-06-01 00:00:00'),
  'Matadero Madrid',
  1),
 ('Acompañamiento digital a personas mayores',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=5b58d89ad2232910VgnVCM2000001f4a900aRCRD',
  None,
  None,
  None,
  Timestamp('2024-10-14 00:00:00'),
  Timestamp('2025-06-30 00:00:00'),
  None,
  1),
 ('Acompañar en la pérdida',

In [29]:
# Creamos la query de inserción de los datos de eventos
insert_query_eventos = """ 
                        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario,
                        fecha_inicio, fecha_fin,organizacion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [30]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_eventos, data_insert_eventos)
conn.commit()

In [31]:
# vamos a comprobar que se ha creado correctamente
query_eventos = """
    SELECT * 
    FROM eventos; 
"""
cur.execute(query_eventos)
cur.fetchmany(3)

[(1,
  '60 Premio Reina Sofía de Pintura y Escultura',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD',
  28009,
  'PASEO COLOMBIA 1',
  None,
  datetime.date(2025, 2, 27),
  datetime.date(2025, 3, 23),
  'Centro Cultural Casa de Vacas (Retiro)',
  1),
 (2,
  'A toda vela',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=59d5e7da8b822910VgnVCM1000001d4a900aRCRD',
  28045,
  'PLAZA LEGAZPI 8',
  None,
  datetime.date(2024, 10, 1),
  datetime.date(2025, 6, 1),
  'Matadero Madrid',
  1),
 (3,
  'Acompañamiento digital a personas mayores',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=5b58d89ad2232910VgnVCM2000001f4a900aRCRD',
  None,
  None,
  None,
  datetime.date(2024, 10, 14),
  datetime.date(2025, 6, 30),
  None,
  1)]

# NO VOLVER A EJECUTAR LA QUERY DE EVENTOS, ESTA YA INSERTADA

### Tabla hoteles

In [32]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'nombre_hotel', 'ciudad', 'precio_noche', 'estrellas'],
      dtype='object')

In [33]:
data_insert_hotels = []

for _, row in data.iterrows():
    id_hotel = row["id_hotel"]
    nombre_hotel = row["nombre_hotel"] 
    competencia = row["competencia"] 
    valoracion = row["estrellas"] 
    id_ciudad = dictio_ciudad.get("Madrid") 

    tupla_hotel = (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)

    if tupla_hotel not in data_insert_hotels:
        data_insert_hotels.append(tupla_hotel)

In [34]:
data_insert_hotels

[('1', 'Hotel Monte Verde', False, 3.1, 1),
 ('2', 'Hotel Brisas del Mar', False, 3.09, 1),
 ('3', 'Hotel Camino del Sol', False, 3.07, 1),
 ('4', 'Hotel Puerta del Cielo', False, 3.03, 1),
 ('5', 'Hotel Encanto Real', False, 3.03, 1),
 ('6', 'Palacio del Sol', False, 2.99, 1),
 ('7', 'Hotel Jardines del Rey', False, 2.94, 1),
 ('8', 'Hotel Las Estrellas', False, 2.91, 1),
 ('9', 'Gran Hotel Madrid', False, 3.07, 1),
 ('10', 'Hotel Torre Dorada', False, 2.92, 1),
 ('11', 'Hotel Palacio Imperial', False, 3.01, 1),
 ('12', 'Hotel Luz de Madrid', False, 3.05, 1),
 ('13', 'Hotel Los Almendros', False, 3.01, 1),
 ('14', 'Hotel Sol y Luna', False, 3.01, 1),
 ('15', 'Hotel Mirador Real', False, 2.98, 1),
 ('16', 'Hotel Rincón Sereno', False, 3.0, 1),
 ('17', 'Hotel Vista Alegre', False, 2.94, 1),
 ('18', 'Hotel Costa Azul', False, 3.12, 1),
 ('19', 'Hotel Maravilla Real', False, 2.98, 1),
 ('20', 'ibis Styles Madrid Prado', True, 4.7, 1),
 ('21', 'ibis Styles Madrid Centro Maravillas', True, 

In [35]:
# Creamos la query de inserción de los datos de ciudad
insert_query_hoteles = """ 
                        INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s)
"""

In [36]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_hoteles, data_insert_hotels)
conn.commit()

In [37]:
# vamos a comprobar que se ha creado correctamente
query_hoteles = """
    SELECT * 
    FROM hoteles; 
"""
cur.execute(query_hoteles)
cur.fetchmany(3)

[('1', 'Hotel Monte Verde', False, 3.1, 1),
 ('2', 'Hotel Brisas del Mar', False, 3.09, 1),
 ('3', 'Hotel Camino del Sol', False, 3.07, 1)]

# HOTELES ESTA PERFECTO NO TOCAR MAS

### Tabla clientes

In [38]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'nombre_hotel', 'ciudad', 'precio_noche', 'estrellas'],
      dtype='object')

In [39]:
data_insert_clientes = []

for _, row in data.iterrows():

    id_cliente = row["id_cliente"]
    nombre = row["nombre"],
    apellido = row["apellido"],
    mail = row["mail"]

    tupla_cliente = (id_cliente, nombre, apellido, mail)

    if tupla_cliente not in data_insert_clientes:
        data_insert_clientes.append(tupla_cliente)

In [40]:
# Creamos la query de inserción de los datos de ciudad
insert_query_clientes = """ 
                        INSERT INTO clientes (id_cliente, nombre, apellido, mail)
                        VALUES (%s, %s, %s, %s)
"""

In [41]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_clientes, data_insert_clientes)
conn.commit()

In [33]:
# vamos a comprobar que se ha creado correctamente
query_clientes = """
    SELECT * 
    FROM clientes; 
"""
cur.execute(query_clientes)
cur.fetchmany(3)

[('1', 'Maite', 'Calatayud', 'maite.calatayud@example.com'),
 ('2', 'Tecla', 'Bonet', 'tecla.bonet@example.com'),
 ('3', 'Amílcar', 'Andrés', 'amílcar.andrés@example.com')]

### Tabla reservas

In [42]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'nombre_hotel', 'ciudad', 'precio_noche', 'estrellas'],
      dtype='object')

In [43]:
# Sacamos el id de clientes de la tabla de clientes
cur.execute("SELECT mail, id_cliente FROM clientes")
dictio_clientes = dict(cur.fetchall())
dictio_clientes

{'maite.calatayud@example.com': '1',
 'tecla.bonet@example.com': '2',
 'amílcar.andrés@example.com': '3',
 'joan.vazquez@example.com': '4',
 'chelo.flor@example.com': '5',
 'dolores.lobo@example.com': '6',
 'noé.lobo@example.com': '7',
 'omar.lamas@example.com': '8',
 'ágata.pinedo@example.com': '9',
 'julie.baños@example.com': '10',
 'javier.mancebo@example.com': '11',
 'rita.guerrero@example.com': '12',
 'ricardo.lobo@example.com': '13',
 'perlita.patiño@example.com': '14',
 'pedro.gallego@example.com': '15',
 'judith.bellido@example.com': '16',
 'francisco javier.hierro@example.com': '17',
 'reynaldo.lerma@example.com': '18',
 'felicidad.gimenez@example.com': '19',
 'pía.barba@example.com': '20',
 'amílcar.moles@example.com': '21',
 'blas.galván@example.com': '22',
 'claudia.miranda@example.com': '23',
 'fernanda.segarra@example.com': '24',
 'emilio.huerta@example.com': '25',
 'bárbara.jódar@example.com': '26',
 'emiliano.vergara@example.com': '27',
 'marisol.arévalo@example.com': '

In [44]:
# Sacamos el id de hotel de la tabla de hoteles
cur.execute("SELECT nombre_hotel, id_hotel FROM hoteles")
dictio_hoteles = dict(cur.fetchall())
dictio_hoteles

{'Hotel Monte Verde': '1',
 'Hotel Brisas del Mar': '2',
 'Hotel Camino del Sol': '3',
 'Hotel Puerta del Cielo': '4',
 'Hotel Encanto Real': '5',
 'Palacio del Sol': '6',
 'Hotel Jardines del Rey': '7',
 'Hotel Las Estrellas': '8',
 'Gran Hotel Madrid': '9',
 'Hotel Torre Dorada': '10',
 'Hotel Palacio Imperial': '11',
 'Hotel Luz de Madrid': '12',
 'Hotel Los Almendros': '13',
 'Hotel Sol y Luna': '14',
 'Hotel Mirador Real': '15',
 'Hotel Rincón Sereno': '16',
 'Hotel Vista Alegre': '17',
 'Hotel Costa Azul': '18',
 'Hotel Maravilla Real': '19',
 'ibis Styles Madrid Prado': '20',
 'ibis Styles Madrid Centro Maravillas': '21',
 'ibis Madrid Centro las Ventas': '22',
 'ibis budget Madrid Centro las Ventas': '23',
 'ibis Styles Madrid City Las Ventas': '24',
 'ibis budget Madrid Calle Alcalá': '25',
 'ibis Madrid Calle Alcalá': '26',
 'ibis budget Madrid Vallecas': '27',
 'ibis Madrid Getafe': '28',
 'ibis Madrid Aeropuerto Barajas': '29'}

In [45]:
print(dictio_hoteles.keys())  # Verifica que la clave realmente está en el diccionario
print(dictio_clientes.keys())

dict_keys(['Hotel Monte Verde', 'Hotel Brisas del Mar', 'Hotel Camino del Sol', 'Hotel Puerta del Cielo', 'Hotel Encanto Real', 'Palacio del Sol', 'Hotel Jardines del Rey', 'Hotel Las Estrellas', 'Gran Hotel Madrid', 'Hotel Torre Dorada', 'Hotel Palacio Imperial', 'Hotel Luz de Madrid', 'Hotel Los Almendros', 'Hotel Sol y Luna', 'Hotel Mirador Real', 'Hotel Rincón Sereno', 'Hotel Vista Alegre', 'Hotel Costa Azul', 'Hotel Maravilla Real', 'ibis Styles Madrid Prado', 'ibis Styles Madrid Centro Maravillas', 'ibis Madrid Centro las Ventas', 'ibis budget Madrid Centro las Ventas', 'ibis Styles Madrid City Las Ventas', 'ibis budget Madrid Calle Alcalá', 'ibis Madrid Calle Alcalá', 'ibis budget Madrid Vallecas', 'ibis Madrid Getafe', 'ibis Madrid Aeropuerto Barajas'])
dict_keys(['maite.calatayud@example.com', 'tecla.bonet@example.com', 'amílcar.andrés@example.com', 'joan.vazquez@example.com', 'chelo.flor@example.com', 'dolores.lobo@example.com', 'noé.lobo@example.com', 'omar.lamas@example.com

In [46]:
data_insert_reservas = []

for _, row in data.iterrows():
    id_reserva = row["id_reserva"]
    fecha_reserva = row["fecha_reserva"]
    inicio_estancia = row["inicio_estancia"]
    final_estancia = row["final_estancia"]
    precio_noche = row["precio_noche"]
    id_cliente = dictio_clientes.get(row["mail"])
    id_hotel = dictio_hoteles.get(row["nombre_hotel"])

    
    tupla_reserva = (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)

    if tupla_reserva not in data_insert_reservas:
        data_insert_reservas.append(tupla_reserva)


In [47]:
data_insert_reservas

[('40c4cb55-d1f5-407b-832f-4756b8ff77b4',
  Timestamp('2025-02-09 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-02 00:00:00'),
  276.61238095238093,
  '1',
  '1'),
 ('f2ce8df5-7844-43e1-8c0f-97ce7a208f21',
  Timestamp('2025-02-08 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-02 00:00:00'),
  275.2259210526316,
  '2',
  '2'),
 ('57d4515a-447d-4067-afcb-9bdcf4e4e915',
  Timestamp('2025-02-02 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-02 00:00:00'),
  269.99844357976656,
  '3',
  '3'),
 ('ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7',
  Timestamp('2025-02-08 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-02 00:00:00'),
  280.1512428298279,
  '4',
  '4'),
 ('99c3dc4f-663c-45f7-849a-ac9313f3746a',
  Timestamp('2025-02-09 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-02 00:00:00'),
  278.2439960629921,
  '5',
  '5'),
 ('a5da0906-0fe5-4c30-b193-863df67a7b84',
  Timestamp('2025-02-10 00

In [48]:
# Creamos la query de inserción de los datos de ciudad
insert_query_reservas = """ 
                        INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [49]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_reservas, data_insert_reservas)
conn.commit()

In [50]:
# vamos a comprobar que se ha creado correctamente
query_reservas = """
    SELECT * 
    FROM reservas; 
"""
cur.execute(query_reservas)
cur.fetchmany(3)

[('40c4cb55-d1f5-407b-832f-4756b8ff77b4',
  datetime.date(2025, 2, 9),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  276.61238095238093,
  '1',
  '1'),
 ('f2ce8df5-7844-43e1-8c0f-97ce7a208f21',
  datetime.date(2025, 2, 8),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  275.2259210526316,
  '2',
  '2'),
 ('57d4515a-447d-4067-afcb-9bdcf4e4e915',
  datetime.date(2025, 2, 2),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  269.99844357976656,
  '3',
  '3')]

## Cierre de la conexión

In [51]:
# una vez hemos terminado de trabajar es necesario cerrar la conexión y el cursor.
cur.close()
conn.close()

## Funciones de carga

In [20]:
# funcion para crear la conexion con la base
def conexion_BBDD(nombre_BBDD = "BBDD_Hoteles", usuario = "postgres", contraseña = "admin", anfitrion = "localhost", puerto = "5432"):
    conn = ps.connect(
        dbname = nombre_BBDD, # base a la que nos queremos conectar
        user = usuario,
        password = contraseña,
        host = anfitrion,
        port = puerto)
    
    cur = conn.cursor()
    cur.execute("SELECT version();")
    return cur.fetchone()

In [15]:
# Carga tabla ciudad
def carga_tabla_ciudad(ciudad = "Madrid"):
    data_to_insert = [ciudad]
    insert_query = """ 
                        INSERT INTO ciudad (nombre_ciudad)
                        VALUES (%s) 
                    """
    cur.execute(insert_query, data_to_insert)
    conn.commit()
    query = """
                    SELECT * 
                    FROM ciudad; 
                """
    cur.execute(query)
    return cur.fetchall()

In [14]:
# Carga tabla eventos
# la query será tal que así: "SELECT nombre_ciudad, id_ciudad FROM ciudad"
def carga_tabla_eventos(dataframe, query_ciudad):
    cur.execute(query_ciudad)
    dictio = dict(cur.fetchall())

    data_to_insert = []


    for i, row in dataframe.iterrows():
        nombre_evento = row["nombre_evento"]
        url_evento = row["url_evento"]
        codigo_postal = row["codigo_postal"] if row["codigo_postal"] != 0 else None # en este caso los valores que son igual a 0 se corresponden con valores nulos, por lo que le indicamos que si el valor es igual a 0 nos ponga un nulo
        direccion = row["direccion"] if pd.notna(row["direccion"]) else None # tenemos valores nulos en esta columna, de manera que le indicamos, que si el elemento no es nulo nos lo coja, y si es nulo, nos incluya un None, ya que sql no acepta valores nulos que no sean None
        horario = row["horario"] if pd.notna(row["horario"]) else None
        fecha_inicio = row["fecha_inicio"]
        fecha_fin = row["fecha_fin"]
        organizacion = row["organizacion"] if pd.notna(row["organizacion"]) else None
        id_ciudad = dictio.get("Madrid")

        data_to_insert.append((nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad))
        
    insert_query = """ 
                    INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario,
                    fecha_inicio, fecha_fin,organizacion, id_ciudad)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """
    cur.executemany(insert_query, data_to_insert)
    conn.commit()

    query = """
                    SELECT * 
                    FROM eventos; 
                """
    cur.execute(query)
    return cur.fetchmany(3)

In [13]:
# Carga tabla hoteles
# la query será tal que así: "SELECT nombre_ciudad, id_ciudad FROM ciudad"
def carga_tabla_hoteles(dataframe, query_ciudad):
    data_to_insert = []

    cur.execute(query_ciudad)
    dictio = dict(cur.fetchall())

    for _, row in dataframe.iterrows():
        id_hotel = row["id_hotel"]
        nombre_hotel = row["nombre_hotel"] 
        competencia = row["competencia"] 
        valoracion = row["estrellas"] 
        id_ciudad = dictio.get("Madrid") 

        tupla = (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)

        if tupla not in data_to_insert:
            data_to_insert.append(tupla)
    
    insert_query = """ 
                        INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s)
            """
    cur.executemany(insert_query, data_to_insert)
    conn.commit()

    query = """
    SELECT * 
    FROM hoteles; 
        """
    cur.execute(query)
    return cur.fetchmany(3)

In [12]:
# Carga tabla clientes
def carga_tabla_clientes(dataframe):
    data_to_insert = []

    for _, row in dataframe.iterrows():

        id_cliente = row["id_cliente"]
        nombre = row["nombre"],
        apellido = row["apellido"],
        mail = row["mail"]

        tupla = (id_cliente, nombre, apellido, mail)

        if tupla_cliente not in data_to_insert:
            data_to_insert.append(tupla)
    
    insert_query = """ 
                        INSERT INTO clientes (id_cliente, nombre, apellido, mail)
                        VALUES (%s, %s, %s, %s)
                        """
    cur.executemany(insert_query, data_to_insert)
    conn.commit()

    query = """
                    SELECT * 
                    FROM clientes; 
                """
    cur.execute(query)
    return cur.fetchmany(3)

In [11]:
# Carga tabla reservas
# La query de clientes será así: "SELECT nombre, id_cliente FROM clientes"
# la query de hoteles será así: "SELECT nombre_hotel, id_hotel FROM hoteles"
def carga_tabla_reservas(dataframe, query_clientes, query_hoteles):
    
    cur.execute(query_clientes)
    dictio_clientes = dict(cur.fetchall())

    cur.execute(query_hoteles)
    dictio_hoteles = dict(cur.fetchall())

    data_to_insert = []

    for _, row in dataframe.iterrows():
        id_reserva = row["id_reserva"]
        fecha_reserva = row["fecha_reserva"]
        inicio_estancia = row["inicio_estancia"]
        final_estancia = row["final_estancia"]
        precio_noche = row["precio_noche"]
        id_cliente = dictio_clientes.get(row["nombre"])
        id_hotel = dictio_hoteles.get(row["nombre_hotel"])

        data_to_insert.append((id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel))
    
    insert_query = """ 
                        INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
                        """
    
    cur.executemany(insert_query, data_to_insert)
    conn.commit()

    query = """
                    SELECT * 
                    FROM reservas; 
                    """
    cur.execute(query)
    cur.fetchmany(3)